# Deadweight Tester Corrected Pressure Calculation

Calculates the corrected pressure using the formula provided in the [OM20594 Deadweight Tester User Manual](./docs/OM20594-user-manual-deadweight-tester.pdf): Temperature and Gravity Corrections.

The corrected pressure is calculated based on the actual pressure, local temperature, calibrated temperature for the deadweight tester, calibrated gravitational acceleration for the deadweight tester (here, standard gravitational acceleration), and local gravitational acceleration.

In [46]:
import math

# Constants
TEMPERATURE_CORRECTION_COEFFICIENT: float = 0.000025  # 0.000025 is a standard value (see deadweight tester calibration sheet if available)
STANDARD_GRAVITATIONAL_ACCELERATION: float = 9.80665  # [m/s^2]
EQUATORIAL_GRAVITATIONAL_ACCELERATION: float = 9.7803184  # [m/s^2]


def calculate_gravitational_acceleration(L: float, H: float) -> float:
    """
    Calculate the gravitational acceleration given the latitude and height
    above sea level using the international gravity formula (WGS-84).
    
    9.7803184 * (1 + 0.0053024 * sin^2(L) - 0.0000059 * sin^2(2L)) - 0.000003086 * H

    Parameters:
    L (float): The latitude [deg]
    H (float): The height above sea level [m]

    Returns:
    float: The local gravitational acceleration [m/s^2]
    """
    # Convert latitude to radians (radians is used by the math library, instead of degrees).
    latitude_rad: float = math.radians(L)

    # Calculate sin^2(L) and sin^2(2L) in advance so the formula below is easier to read.
    sin2_L: float = math.sin(latitude_rad)**2
    sin2_2L: float = math.sin(2 * latitude_rad)**2

    # Calculate the gravitational acceleration.
    # (9.7803184 is the equatorial gravitational acceleration at sea level.)
    g: float = EQUATORIAL_GRAVITATIONAL_ACCELERATION * (1 + 0.0053024 * sin2_L - 0.0000059 * sin2_2L) - 0.000003086 * H

    return g


def calculate_corrected_pressure(P2: float, t: float, T: float, g: float, G: float) -> float:
    """
    Calculate the corrected pressure using the formula described in the OM20594 manual.

    P1 = P2 * (1 + 0.000025 * (T - t)) * (g / G)

    Parameters:
    P2 (float): The actual pressure [bar]
    t (float): The temperature at the position of the deadweight tester [deg C]
    T (float): The calibrated temperature for the deadweight tester [deg C]
    g (float): The local gravitational acceleration [m/s^2]
    G (float): The calibrated gravitational acceleration for the deadweight tester [m/s^2]

    Returns:
    float: The corrected pressure [bar]
    """
    
    return P2 * (1 + TEMPERATURE_CORRECTION_COEFFICIENT * (T - t)) * (g / G)


# Input values for local gravitational acceleration:
L = 60.3968978  # latitude [deg] (Rozenkrantztårnet, Bergen, Norway)
H = 16  # height above sea level [m]

# Input values for corrected pressure:
P2: float = 601.007  # Actual pressure [bar] (600.000 bar weight plus 1.007 bar atmospheric pressure)
t: float = 19.9  # Temperature at the position of the deadweight tester [deg C]
T: float = 20.0  # Calibrated temperature for deadweight tester [deg C]
g: float = calculate_gravitational_acceleration(L=L, H=H)  # Local gravitational acceleration [m/s^2]
G: float = STANDARD_GRAVITATIONAL_ACCELERATION  # Calibrated gravitational acceleration for the deadweight tester [m/s^2]

# Calculate the corrected pressure
P1 = calculate_corrected_pressure(
    P2=P2,
    t=t,
    T=T,
    g=g,
    G=G
)

# Formatted printing for clarity
def print_formatted(name: str, value: any, unit: str, description: str) -> None:
    print(f"{name:<10} {value:<20} {unit:<10} {description}")

# Output the input values and the result
print_formatted("Name", "Value", "Unit", "Description")
print("-" * 110)
print_formatted("L", f"{L}", "deg", "Geographical latitude")
print_formatted("H", f"{H}", "m", "Height above sea level")
print("-" * 110)
print_formatted("P2", f"{P2}", "bar", "Actual pressure")
print_formatted("t", f"{t}", "deg C", "Temperature at the position of the deadweight tester")
print_formatted("T", f"{T}", "deg C", "Calibrated temperature for the deadweight tester")
print_formatted("g", f"{g}", "m/s^2", "Local gravitational acceleration")
print_formatted("G", f"{G}", "m/s^2", "Calibrated gravitational acceleration for the deadweight tester")
print("-" * 110)
print_formatted("P1", f"{P1}", "bar", "Corrected pressure")

Name       Value                Unit       Description
--------------------------------------------------------------------------------------------------------------
L          60.3968978           deg        Geographical latitude
H          16                   m          Height above sea level
--------------------------------------------------------------------------------------------------------------
P2         601.007              bar        Actual pressure
t          19.9                 deg C      Temperature at the position of the deadweight tester
T          20.0                 deg C      Calibrated temperature for the deadweight tester
g          9.819430668625222    m/s^2      Local gravitational acceleration
G          9.80665              m/s^2      Calibrated gravitational acceleration for the deadweight tester
--------------------------------------------------------------------------------------------------------------
P1         601.7917761646288    bar        Correcte